In [295]:
map = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L"""

In [296]:
def parseMap(parseMap):
    lines = parseMap.splitlines()
    for line in lines:
        objectA, objectB = line.split(")")
        yield objectA, objectB

In [297]:
for objectAName, objectBName in parseMap(map):
    print(objectAName, objectBName)

COM B
B C
C D
D E
E F
B G
G H
D I
E J
J K
K L


In [298]:
from math import inf

In [320]:
class SpaceObject:
    def __init__(self, name):
        self.name = name
        self.children = []
        self.parentObject = None
        
        # dijkstra
        self.visited = False
        self.distance = inf

    def append(self, spaceObject):
        spaceObject.parentObject = self
        self.children.append(spaceObject)

    def __repr__(self):
        if self.parentObject:
            return "[%s, parent=%s, children=%d]" % \
              (self.name, self.parentObject.name, len(self.children))
        else:
            return "[%s, children=%d]" % (self.name, len(self.children))

In [39]:
orbitMap = SpaceObject("COM")

In [40]:
orbitMap

[COM, children=0]

In [41]:
def findObject(orbitMap, name):
    if orbitMap.name == name:
        return orbitMap

    for childObject in orbitMap.children:
        result = findObject(childObject, name)
        if result:
            return result

    return None

In [42]:
findObject(orbitMap, "COM").name

'COM'

In [264]:
def insertObject(orbitMap, spaceObject, parentName):
    parent = findObject(orbitMap, parentName)
    assert(parent != None)
    parent.append(spaceObject)

In [46]:
for parentName, childName in parseMap(map):
    spaceObject = SpaceObject(childName)
    insertObject(orbitMap, spaceObject, parentName)

In [47]:
orbitMap

[COM, children=1]

In [51]:
D = findObject(orbitMap, "D")

In [52]:
D

[D, parent=C, children=2]

In [54]:
def countOrbits(spaceObject):
    if spaceObject.parentObject:
        return 1 + countOrbits(spaceObject.parentObject)
    else:
        return 0

In [55]:
countOrbits(D)

3

In [56]:
L = findObject(orbitMap, "L")

In [57]:
L

[L, parent=K, children=0]

In [58]:
countOrbits(L)

7

In [61]:
COM = findObject(orbitMap, "COM")

In [62]:
COM

[COM, children=1]

In [166]:
countOrbits(COM)

0

In [170]:
def listObjects(orbitMap):
    result = [orbitMap]
    for child in orbitMap.children:
        result = result + listObjects(child)
    return result

In [172]:
listObjects(COM)

[[COM, children=1],
 [B, parent=COM, children=2],
 [C, parent=B, children=1],
 [D, parent=C, children=2],
 [E, parent=D, children=2],
 [F, parent=E, children=0],
 [J, parent=E, children=1],
 [K, parent=J, children=1],
 [L, parent=K, children=0],
 [I, parent=D, children=0],
 [G, parent=B, children=1],
 [H, parent=G, children=0]]

In [174]:
def computeTotalOrbits(orbitMap):
    result = 0
    for spaceObject in listObjects(orbitMap):
        result += countOrbits(spaceObject)
    return result

In [176]:
computeTotalOrbits(COM)

42

In [202]:
import pandas as pd

In [359]:
df = pd.read_csv("input", delimiter=")", header=None)

In [360]:
df.columns = ["parent", "child"]

In [361]:
nodes = []

In [362]:
vertices = {}

In [363]:
for mapping in df.itertuples():
    if not mapping.parent in nodes:
        nodes.append(mapping.parent)

    if not mapping.child in nodes:
        nodes.append(mapping.child)        
        
    if mapping.parent in vertices:
        vertices[mapping.parent].append(mapping.child)
    else:
        vertices[mapping.parent] = [mapping.child]

In [364]:
len(nodes)

2307

In [365]:
len(vertices)

2097

In [366]:
root = None

In [367]:
for node in nodes:
    found = False

    for children in vertices.values():
        if node in children:
            found = True
            continue
            
    if not found:
        root = node

In [368]:
root

'COM'

In [382]:
rootObject = SpaceObject("COM")

In [383]:
def constructObjectMap(parentObject, nodes, vertices):
    if parentObject.name in vertices:
        for child in vertices[parentObject.name]:
            childObject = SpaceObject(child)
            constructObjectMap(childObject, nodes, vertices)
            parentObject.append(childObject)

In [384]:
constructObjectMap(rootObject, nodes, vertices)

In [385]:
rootObject

[COM, children=1]

In [373]:
len(listObjects(rootObject))

2307

In [374]:
computeTotalOrbits(rootObject)

453028

In [303]:
map = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN"""

In [304]:
orbitMap = SpaceObject("COM")

In [305]:
for parentName, childName in parseMap(map):
    spaceObject = SpaceObject(childName)
    insertObject(orbitMap, spaceObject, parentName)

In [306]:
orbitMap

[COM, children=1]

In [307]:
YOU = findObject(orbitMap, "YOU")

In [308]:
YOU

[YOU, parent=K, children=0]

In [309]:
SAN = findObject(orbitMap, "SAN")

In [310]:
SAN

[SAN, parent=I, children=0]

In [336]:
def unvisitedNeighbors(spaceObject):
    neighbors = []
    if spaceObject.parentObject and not spaceObject.parentObject.visited:
        neighbors.append(spaceObject.parentObject)
    for childObject in spaceObject.children:
        if not childObject.visited:
            neighbors.append(childObject)
    return neighbors

In [337]:
def dijkstra(current, unvisited, SAN):
    tentativeDistance = current.distance + 1
    for neighbor in unvisitedNeighbors(current):
        neighbor.distance = min(tentativeDistance, neighbor.distance)
    current.visited = True
    unvisited.remove(current)
    if current.name == SAN.name:
        return current.distance
    else:
        smallestTentativeDistance = inf
        nextCurrent = None
        for candidate in unvisited:
            if candidate.distance < smallestTentativeDistance:
                smallestTentativeDistance = candidate.distance
                nextCurrent = candidate
        return dijkstra(nextCurrent, unvisited, SAN)

In [346]:
unvisited = listObjects(orbitMap)

In [347]:
YOU.distance = 0

In [348]:
dijkstra(YOU, unvisited, SAN) - 2

4

In [386]:
YOU = findObject(rootObject, "YOU")

In [387]:
YOU

[YOU, parent=524, children=0]

In [388]:
SAN = findObject(rootObject, "SAN")

In [389]:
SAN

[SAN, parent=8N9, children=0]

In [393]:
unvisited = listObjects(rootObject)

In [394]:
YOU.distance = 0

In [395]:
dijkstra(YOU, unvisited, SAN) - 2

562